# Data Manipulation with Pandas

Pandas is a newer package built on top of NumPy, and provides an efficient implementation of a DataFrame. DataFrames are essentially multidimensional arrays with attached row and column labels, and often with heterogeneous types and/or missing data. 

As well as offering a convenient storage interface for labeled data, Pandas implements a number of powerful data operations familiar to users of both database frameworks and spreadsheet programs.

As we saw, NumPy's ndarray data structure provides essential features for the type of clean, well-organized data typically seen in numerical computing tasks.

While it serves this purpose very well, its limitations become clear when we need more flexibility (e.g., attaching labels to data, working with missing data, etc.) and when attempting operations that do not map well to element-wise broadcasting (e.g., groupings, pivots, etc.), each of which is an important piece of analyzing the less structured data available in many forms in the world around us.

Pandas, and in particular its Series and DataFrame objects, builds on the NumPy array structure and provides efficient access to these sorts of "data munging" tasks that occupy much of a data scientist's time.

In this section, we will focus on the mechanics of using Series, DataFrame, and related structures effectively. We will use examples drawn from real datasets where appropriate, but these examples are not necessarily the focus.

# 1. Introduction to the Pandas Object

At the very basic level, Pandas objects can be thought of as enhanced versions of NumPy structured arrays in which the rows and columns are identified with labels rather than simple integer indices. 

As we will see during the course of this chapter, Pandas provides a host of useful tools, methods, and functionality on top of the basic data structures, but nearly everything that follows will require an understanding of what these structures are. Thus, before we go any further, let's introduce these three fundamental Pandas data structures: the Series, DataFrame, and Index.


Just as we generally import NumPy under the alias np, we will import Pandas under the alias pd:

In [1]:
import numpy as np
import pandas as pd

### The Pandas Series Object

A Pandas Series is a one-dimensional array of indexed data. It can be created from a list or array as follows:

In [2]:
data = pd.Series([0.25, 0.5, 0.75, 1.0])
data

0    0.25
1    0.50
2    0.75
3    1.00
dtype: float64


As we see in the output, the Series wraps both a sequence of values and a sequence of indices, which we can access with the values and index attributes. The values are simply a familiar NumPy array:

In [3]:
data.values

array([0.25, 0.5 , 0.75, 1.  ])

The index is an array-like object of type pd.Index, which we'll discuss in more detail momentarily.

In [4]:
data.index

RangeIndex(start=0, stop=4, step=1)

ike with a NumPy array, data can be accessed by the associated index via the familiar Python square-bracket notation:

In [5]:
data[1]

0.5

In [6]:
data[1:3]

1    0.50
2    0.75
dtype: float64


As we will see, though, the Pandas Series is much more general and flexible than the one-dimensional NumPy array that it emulates.

### Series as generalized NumPy array



From what we've seen so far, it may look like the Series object is basically interchangeable with a one-dimensional NumPy array. The essential difference is **the presence of the index**: while the Numpy Array has an implicitly defined integer index used to access the values, the Pandas Series has an explicitly defined index associated with the values.

This explicit index definition gives the Series object additional capabilities. For example, the index need not be an integer, but can consist of values of any desired type. For example, if we wish, we can use strings as an index:

In [7]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [8]:
# And the item access works as expected:
data['a']

0.25

We can even use non-contiguous or non-sequential indices:

In [9]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=[2, 5, 3, 7])
data

2    0.25
5    0.50
3    0.75
7    1.00
dtype: float64

In [10]:
data[5]

0.5

### Series as specialized dictionary

In this way, you can think of a Pandas Series a bit like a specialization of a Python dictionary. 

A dictionary is a structure that maps arbitrary keys to a set of arbitrary values, and a Series is a structure which maps typed keys to a set of typed values. 

This typing is important: just as the type-specific compiled code behind a NumPy array makes it more efficient than a Python list for certain operations, the type information of a Pandas Series makes it much more efficient than Python dictionaries for certain operations.

The Series-as-dictionary analogy can be made even more clear by constructing a Series object directly from a Python dictionary:

In [11]:
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}

population = pd.Series(population_dict)
population

California    38332521
Florida       19552860
Illinois      12882135
New York      19651127
Texas         26448193
dtype: int64

By default, a Series will be created where the index is drawn from the sorted keys. From here, typical dictionary-style item access can be performed:

In [12]:
population['California']

38332521

Unlike a dictionary, though, the Series also supports array-style operations such as slicing:

In [13]:
population['California':'Florida']

California    38332521
Florida       19552860
dtype: int64

### Constructing Series objects


We've already seen a few ways of constructing a Pandas Series from scratch; all of them are some version of the following:
> pd.Series(data, index=index)

where index is an optional argument, and data can be one of many entities.

For example, data can be a list or NumPy array, in which case index defaults to an integer sequence:

In [14]:
pd.Series([2, 4, 6])

0    2
1    4
2    6
dtype: int64

data can be a scalar, which is repeated to fill the specified index:

In [15]:
pd.Series(5, index=[100, 200, 300])

100    5
200    5
300    5
dtype: int64


data can be a dictionary, in which index defaults to the sorted dictionary keys:

In [16]:
pd.Series({2:'a', 1:'b', 3:'c'})

1    b
2    a
3    c
dtype: object

In each case, the index can be explicitly set if a different result is preferred:

In [17]:
pd.Series({2:'a', 1:'b', 3:'c'}, index=[3, 2])

3    c
2    a
dtype: object

## The Pandas DataFrame Object


The next fundamental structure in Pandas is the DataFrame. Like the Series object discussed in the previous section, the DataFrame can be thought of either as a generalization of a NumPy array, or as a specialization of a Python dictionary. We'll now take a look at each of these perspectives.

### DataFrame as a generalized NumPy array


If a Series is an analog of a one-dimensional array with flexible indices, a DataFrame is an analog of a two-dimensional array with both flexible row indices and flexible column names. 

Just as you might think of a two-dimensional array as an ordered sequence of aligned one-dimensional columns, you can think of a DataFrame as a sequence of aligned Series objects. Here, by "aligned" we mean that they share the same index.

To demonstrate this, let's first construct a new Series listing the area of each of the five states discussed in the previous section:

In [18]:
area_dict = {'California': 423967, 'Texas':6956622, 'New York': 141297,
            'Florida': 170312, 'Illinois': 149995}

area = pd.Series(area_dict)
area

California     423967
Florida        170312
Illinois       149995
New York       141297
Texas         6956622
dtype: int64

Now that we have this along with the population Series from before, we can use a dictionary to construct a single two-dimensional object containing this information:

In [19]:
states = pd.DataFrame({'population': population,
                      'area': area})
states

,area,population
California,423967,38332521
Florida,170312,19552860
Illinois,149995,12882135
New York,141297,19651127
Texas,6956622,26448193


Like the Series object, the DataFrame has an index attribute that gives access to the index labels:

In [20]:
states.index

Index(['California', 'Florida', 'Illinois', 'New York', 'Texas'], dtype='object')

Additionally, the DataFrame has a columns attribute, which is an Index object holding the column labels:

In [21]:
states.columns

Index(['area', 'population'], dtype='object')


Thus the DataFrame can be thought of as a generalization of a two-dimensional NumPy array, where both the rows and columns have a generalized index for accessing the data.

### DataFrame as specialized dictionary


Similarly, we can also think of a DataFrame as a specialization of a dictionary.

Where a dictionary maps a key to a value, a DataFrame maps a column name to a Series of column data. 

For example, asking for the 'area' attribute returns the Series object containing the areas we saw earlier:

In [22]:
states['area']

California     423967
Florida        170312
Illinois       149995
New York       141297
Texas         6956622
Name: area, dtype: int64

Notice the potential point of confusion here: in a two-dimesnional NumPy array, data[0] will return the first row. For a DataFrame, data['col0'] will return the first column

### Constructing DataFrame objects

A Pandas DataFrame can be constructed in a variety of ways. Here we'll give several examples.

In [23]:
# From a single Series object

pd.DataFrame(population, columns=['population'])

,population
California,38332521
Florida,19552860
Illinois,12882135
New York,19651127
Texas,26448193


In [24]:
# From a list of dicts

data = [{'a': i, 'b': 2* i} for i in range(3)]
pd.DataFrame(data)

,a,b
0,0,0
1,1,2
2,2,4


In [25]:
# From a two-dimensional NumPy array
pd.DataFrame(np.random.rand(3, 2),
            columns=['foo', 'bar'],
            index=['a', 'b', 'c'])

,foo,bar
a,0.760441,0.987606
b,0.525191,0.796566
c,0.632104,0.792502


In [26]:
# From a NumPy structured array
A = np.zeros((3, 4))
A

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [27]:
pd.DataFrame(A)

,0,1,2,3
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0


## The Pandas Index Object

We have seen here that both the Series and DataFrame objects contain an explicit index that lets you reference and modify data.

This Index object is an interesting structure in itself, and it can be thought of either as an immutable array or as an ordered set (technically a multi-set, as Index objects may contain repeated values).

Those views have some interesting consequences in the operations available on Index objects. As a simple example, let's construct an Index from a list of integers:

In [28]:
ind = pd.Index([2, 3, 5, 7, 11])
ind

Int64Index([2, 3, 5, 7, 11], dtype='int64')

The Index in many ways operates like an array. For example, we can use standard Python indexing notation to retrieve values or slices:

In [29]:
ind[0]

2

In [30]:
ind[::2]

Int64Index([2, 5, 11], dtype='int64')

In [31]:
ind.size

5

In [32]:
ind.shape

(5,)

In [33]:
ind.ndim

1

In [34]:
ind.dtype

dtype('int64')


One difference between Index objects and NumPy arrays is that indices are immutable–that is, they cannot be modified via the normal means:



In [35]:
#ind[1] = 0

This immutability makes it safer to share indices between multiple DataFrames and arrays, without the potential for side effects from inadvertent index modification.

# 2. Data Indexing And Selection

## Indexers: loc and iloc

For example, if your Series has an explicit integer index, an indexing operation such as data[1] will use the explicit indices, while a slicing operation like data[1:3] will use the implicit Python-style index.

In [36]:
data = pd.Series(['a', 'b','c'], index=[1, 3, 5])
data

1    a
3    b
5    c
dtype: object

In [37]:
#  explicit index when indexing
data[1]

'a'

In [38]:
# implicit index when slicing
data[1:3]

3    b
5    c
dtype: object

Because of this potential confusion in the case of integer indexes, Pandas provides some special indexer attributes that explicitly expose certain indexing schemes. These are not functional methods, but attributes that expose a particular slicing interface to the data in the Series.

First, the loc attribute allows indexing and slicing that always references the explicit index:

In [39]:
data.loc[1]

'a'

In [40]:
data.loc[1:3]

1    a
3    b
dtype: object

The iloc attribute allows indexing and slicing that always references the implicit Python-style index:

In [41]:
data.iloc[1]

'b'

In [42]:
data.iloc[1:3]

3    b
5    c
dtype: object

A third indexing attribute, ix, is a hybrid of the two, and for Series objects is equivalent to standard []-based indexing. The purpose of the ix indexer will become more apparent in the context of DataFrame objects, which we will discuss in a moment.

One guiding principle of Python code is that "explicit is better than implicit." 

The explicit nature of loc and iloc make them very useful in maintaining clean and readable code; especially in the case of integer indexes, I recommend using these both to make code easier to read and understand, and to prevent subtle bugs due to the mixed indexing/slicing convention.



### Data Selection in DataFrame

Recall that a DataFrame acts in many ways like a two-dimensional or structured array, and in other ways like a dictionary of Series structures sharing the same index. These analogies can be helpful to keep in mind as we explore data selection within this structure.

In [43]:
data = pd.DataFrame({'pop': population,
             'area': area})

data

,area,pop
California,423967,38332521
Florida,170312,19552860
Illinois,149995,12882135
New York,141297,19651127
Texas,6956622,26448193


The individual Series that make up the columns of the DataFrame can be accessed via dictionary-style indexing of the column name:

In [44]:
data['area']

California     423967
Florida        170312
Illinois       149995
New York       141297
Texas         6956622
Name: area, dtype: int64

Equivalently, we can use attribute-style access with column names that are strings:

In [45]:
data.area

California     423967
Florida        170312
Illinois       149995
New York       141297
Texas         6956622
Name: area, dtype: int64

In [46]:
data['area'] is data.area

True

For example, if the column names are not strings, or if the column names conflict with methods of the DataFrame, this attribute-style access is not possible. For example, the  DataFrame has a pop() method, so data.pop will point to this rather than the "pop" column:

In [47]:
data['pop'] is data.pop

False

In particular, you should avoid the temptation to try column assignment via attribute (i.e., use data['pop'] = z rather than data.pop = z).

This dictionary-style syntax can also be used to modify the object, in this case adding a new column.

In [48]:
data['density'] = data['pop'] / data['area']
data

,area,pop,density
California,423967,38332521,90.413926
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763
New York,141297,19651127,139.076746
Texas,6956622,26448193,3.801873


### DataFrame as two-dimensional array

As mentioned previously, we can also view the DataFrame as an enhanced two-dimensional array. We can examine the raw underlying data array using the values attribute:

In [49]:
data.values

array([[4.23967000e+05, 3.83325210e+07, 9.04139261e+01],
       [1.70312000e+05, 1.95528600e+07, 1.14806121e+02],
       [1.49995000e+05, 1.28821350e+07, 8.58837628e+01],
       [1.41297000e+05, 1.96511270e+07, 1.39076746e+02],
       [6.95662200e+06, 2.64481930e+07, 3.80187295e+00]])

With this picture in mind, many familiar array-like observations can be done on the DataFrame itself. For example, we can transpose the full DataFrame to swap rows and columns:

In [50]:
data.T

,California,Florida,Illinois,New York,Texas
area,4.239670e+05,1.703120e+05,1.499950e+05,1.412970e+05,6.956622e+06
pop,3.833252e+07,1.955286e+07,1.288214e+07,1.965113e+07,2.644819e+07
density,9.041393e+01,1.148061e+02,8.588376e+01,1.390767e+02,3.801873e+00


When it comes to indexing of DataFrame objects, however, it is clear that the dictionary-style indexing of columns precludes our ability to simply treat it as a NumPy array. In particular, passing a single index to an array accesses a row:

In [51]:
data.values[0]

array([4.23967000e+05, 3.83325210e+07, 9.04139261e+01])

and passing a single "index" to a DataFrame accesses a column:

In [52]:
data['area']

California     423967
Florida        170312
Illinois       149995
New York       141297
Texas         6956622
Name: area, dtype: int64

Thus for array-style indexing, we need another convention. Here Pandas again uses the loc, iloc, and ix indexers mentioned earlier. Using the iloc indexer, we can index the underlying array as if it is a simple NumPy array (using the implicit Python-style index), but the DataFrame index and column labels are maintained in the result:

In [53]:
data.iloc[:3, :2]

,area,pop
California,423967,38332521
Florida,170312,19552860
Illinois,149995,12882135


Similarly, using the loc indexer we can index the underlying data in an array-like style but using the explicit index and column names:

In [54]:
data.loc[:'New York', :'pop']

,area,pop
California,423967,38332521
Florida,170312,19552860
Illinois,149995,12882135
New York,141297,19651127


There used to be an ix indexer thad allowd use hybrid of these two approaches , however it is now deprecated so we will skip it.

Any of the familiar NumPy-style data access patterns can be used within these indexers. For example, in the loc indexer we can combine masking and fancy indexing as in the following:

In [55]:
data.loc[data.density > 100, ['pop', 'density']]

,pop,density
Florida,19552860,114.806121
New York,19651127,139.076746


Any of these indexing conventions may also be used to set or modify values; this is done in the standard way that you might be accustomed to from working with NumPy:

In [56]:
data.iloc[0, 2] = 90
data

,area,pop,density
California,423967,38332521,90.000000
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763
New York,141297,19651127,139.076746
Texas,6956622,26448193,3.801873


### Additional indexing conventions

There are a couple extra indexing conventions that might seem at odds with the preceding discussion, but nevertheless can be very useful in practice. First, while indexing refers to columns, slicing refers to rows:

In [57]:
data['Florida':'Illinois']

,area,pop,density
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [58]:
data[1:3]

,area,pop,density
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


Similarly, direct masking operations are also interpreted row-wise rather than column-wise:

In [59]:
data[data.density > 100]

,area,pop,density
Florida,170312,19552860,114.806121
New York,141297,19651127,139.076746


These two conventions are syntactically similar to those on a NumPy array, and while these may not precisely fit the mold of the Pandas conventions, they are nevertheless quite useful in practice.

# 3. Operating on Data in Pandas

Because Pandas is designed to work with NumPy, any NumPy ufunc will work on Pandas Series and DataFrame objects. Let's start by defining a simple Series and DataFrame on which to demonstrate this:

In [60]:
rng = np.random.RandomState(42)
df = pd.DataFrame(rng.randint(0, 10, (3, 4)),
                 columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,3,7,4
1,6,9,2,6
2,7,4,3,7


In [61]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,-0.707107,1.224647e-16
1,-1.000000,7.071068e-01,1.000000,-1.000000e+00
2,-0.707107,1.224647e-16,0.707107,-7.071068e-01


### Missing data

Any item for which one or the other does not have an entry is marked with NaN, or "Not a Number," which is how Pandas marks missing data. 

This index matching is implemented this way for any of Python's built-in arithmetic expressions; any missing values are filled in with NaN by default:

In [62]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

If using NaN values is not the desired behavior, the fill value can be modified using appropriate object methods in place of the operators.

For example, calling A.add(B) is equivalent to calling A + B, but allows optional explicit specification of the fill value for any elements in A or B that might be missing:

In [63]:
A.add(B, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

### Index alignment in DataFrame

In [64]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                columns=list('AB'))
A

,A,B
0,2,1
1,11,5


In [65]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)),
                columns=list('ABC'))
B

,A,B,C
0,1,4,0
1,9,5,8
2,0,9,2


In [66]:
A + B

,A,B,C
0,3.0,5.0,NaN
1,20.0,10.0,NaN
2,NaN,NaN,NaN


As was the case with Series, we can pass any desired fill_value to be used in place of missing entries. Here we'll fill with the mean of all values in A (computed by first stacking the rows of A):

In [67]:
fill = A.stack().mean()
A.add(B, fill_value=fill)

,A,B,C
0,3.00,5.00,4.75
1,20.00,10.00,12.75
2,4.75,13.75,6.75


# 4.  Missing Values

The difference between data found in many tutorials and data in the real world is that real-world data is rarely clean and homogeneous. In particular, many interesting datasets will have some amount of data missing. To make matters even more complicated, different data sources may indicate missing data in different ways.

## NaN and None in Pandas

NaN and None both have their place, and Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate:

In [68]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

For types that don't have an available sentinel value, Pandas automatically type-casts when NA values are present. 

## Operating on Null Values

As we have seen, Pandas treats None and NaN as essentially interchangeable for indicating missing or null values. 

To facilitate this convention, there are several useful methods for detecting, removing, and replacing null values in Pandas data structures

### Detecting null values

Pandas data structures have two useful methods for detecting null data: isnull() and notnull(). Either one will return a Boolean mask over the data. For example:

In [69]:
data = pd.Series([1, np.nan, 'hello', None])

In [70]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [71]:
data[data.notnull()]

0        1
2    hello
dtype: object

### Dropping null values

In [72]:
data.dropna()

0        1
2    hello
dtype: object

We cannot drop single values from a DataFrame; **we can only drop full rows or full columns**. Depending on the application, you might want one or the other, so dropna() gives a number of options for a DataFrame.


In [73]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


By default, dropna() will drop all rows in which any null value is present:

In [74]:
df.dropna()

,0,1,2
1,2.0,3.0,5


In [75]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


But this drops some good data as well; you might rather be interested in dropping rows or columns with all NA values, or a majority of NA values. This can be specified through the how or thresh parameters, which allow fine control of the number of nulls to allow through.



In [76]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [77]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


### Filling null values

Sometimes rather than dropping NA values, you'd rather replace them with a valid value.

In [78]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [79]:
# We can fill NA entries with a single value, such as zero:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [80]:
# We can specify a forward-fill to 
# propagate the previous value forward:
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [81]:
# Or we can specify a back-fill to 
# propagate the next values backward:

data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

# 5. Hierarchical Indexing

Often it is useful to go beyond this and store higher-dimensional data–that is, data indexed by more than one or two keys. 

A common pattern in practice is to make use of hierarchical indexing (also known as multi-indexing) to incorporate multiple index levels within a single index.

In this way, higher-dimensional data can be compactly represented within the familiar one-dimensional Series and two-dimensional DataFrame objects.

In [82]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]

populations = [33871648, 3725395,
              18976457, 19378102,
              20851820, 25145561]

pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)     3725395
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

With this indexing scheme, you can straightforwardly index or slice the series based on this multiple index:

In [83]:
pop[('California', 2010):('Texas', 2010)]

(California, 2010)     3725395
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

## Pandas MultiIndex

Our tuple-based indexing is essentially a rudimentary multi-index, and the Pandas MultiIndex type gives us the type of operations we wish to have. We can create a multi-index from the tuples as follows:

In [84]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

Notice that the MultiIndex contains multiple levels of indexing–in this case, the state names and the years, as well as multiple labels for each data point which encode these levels.

In [85]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010     3725395
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Here the first two columns of the Series representation show the multiple index values, while the third column shows the data. 

Notice that some entries are missing in the first column: in this multi-index representation, any blank entry indicates the same value as the line above it.

In [86]:
# access all data for which the second index is 2010
pop[:, 2010]

California     3725395
New York      19378102
Texas         25145561
dtype: int64

## MultiIndex as extra dimension


You might notice something else here: we could easily have stored the same data using a simple DataFrame with index and column labels. 

In fact, Pandas is built with this equivalence in mind. The unstack() method will quickly convert a multiply indexed  Series into a conventionally indexed DataFrame:

In [87]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,3725395
New York,18976457,19378102
Texas,20851820,25145561


In [88]:
# the stack method provides the opposite operation:
pop_df.stack()

California  2000    33871648
            2010     3725395
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

We can also use hierarchical indexing to represent data of three or more dimensions in a Series or DataFrame.

Each extra level in a multi-index represents an extra dimension of data.

We might want to add another column of demographic data for each state at each year (say, population under 18) ; with a MultiIndex this is as easy as adding another column to the DataFrame:

In [89]:
pop_df = pd.DataFrame({'total': pop,
                     'under18': [9267089, 9284094,
                                467342, 43118033,
                                590631, 6879014]})
pop_df

total   under18
California 2000  33871648   9267089
           2010   3725395   9284094
New York   2000  18976457    467342
           2010  19378102  43118033
Texas      2000  20851820    590631
           2010  25145561   6879014

In [90]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,2.492110
New York,0.024627,2.225091
Texas,0.028325,0.273568


## Indexing and Slicing a MultiIndex

Indexing and slicing on a MultiIndex is designed to be intuitive, and it helps if you think about the indices as added dimensions. We'll first look at indexing multiply indexed Series, and then multiply-indexed DataFrames.

In [91]:
pop

California  2000    33871648
            2010     3725395
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [92]:
# We can access single elements by indexing with multiple terms:
pop['California', 2000]

33871648

The MultiIndex also supports partial indexing, or indexing just one of the levels in the index. 

The result is another Series, with the lower-level indices maintained:

In [93]:
pop['California']

2000    33871648
2010     3725395
dtype: int64

Partial slicing is available as well, as long as the MultiIndex is sorted.

In [94]:
pop.loc['California':'New York']

California  2000    33871648
            2010     3725395
New York    2000    18976457
            2010    19378102
dtype: int64

With sorted indices, partial indexing can be performed on lower levels by passing an empty slice in the first index:

In [95]:
pop[:, 2000]

California    33871648
New York      18976457
Texas         20851820
dtype: int64

# 6. Combining Datasets: Concat and Append

Pandas includes functions and methods that allow combining data from different data sources in a fast and straightforward way.

Here we'll take a look at simple concatenation of Series and DataFrames with the pd.concat function; later we'll dive into more sophisticated in-memory merges and joins implemented in Pandas.

## Simple Concatenation with pd.concat

pd.concat() can be used for a simple concatenation of Series or DataFrame objects, just as np.concatenate() can be used for simple concatenations of arrays:

In [96]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

For convenience, we'll define this function which creates a DataFrame of a particular form that will be useful below:



In [97]:
def make_df(cols, ind):
    data = {c: [str(c) + str(i) for i in ind] for c in cols}
    return pd.DataFrame(data, ind)

make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


In [98]:
df1 = make_df('AB', [1, 2])
df1

,A,B
1,A1,B1
2,A2,B2


In [99]:
df2 = make_df('AB', [3, 4])
df2

,A,B
3,A3,B3
4,A4,B4


In [100]:
pd.concat([df1, df2])

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


By default, the concatenation takes place row-wise within the DataFrame (i.e., axis=0). 
We can specify the axis.

In [101]:
df3 = make_df('CD', [1, 2])
pd.concat([df1, df3], axis='columns')

,A,B,C,D
1,A1,B1,C1,D1
2,A2,B2,C2,D2


## The append() method

Because direct array concatenation is so common, Series and DataFrame objects have an append method that can accomplish the same thing in fewer keystrokes

In [102]:
df1.append(df2)

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


# 7. Combining Datasets: Merge and Join

Pandas also supports in-memory join and merge operations. If you have ever worked with databases, you should be familiar with this type of data interaction. 


Types of joins include:
* One-to-one joins
* Many-to-one joins
* Many-to-Many joins

In [103]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                   'group': ['Accounting', 'Eng', 'Eng', 'HR']})

df1


,employee,group
0,Bob,Accounting
1,Jake,Eng
2,Lisa,Eng
3,Sue,HR


In [104]:
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                   'hire_date': ['2004', '2008', '2012', '2014']})

df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


To combine this information into a single DataFrame, we can use the pd.merge() function:

In [105]:
# one to one join
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Eng,2012
2,Lisa,Eng,2004
3,Sue,HR,2014


The pd.merge() function recognizes that each DataFrame has an "employee" column, and automatically joins using this column as a key.

## Specification of the Merge Key

In [106]:
pd.merge(df1, df2, on='employee')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Eng,2012
2,Lisa,Eng,2004
3,Sue,HR,2014


## The left_on and right_on keywords


At times you may wish to merge two datasets with different column names; for example, we may have a dataset in which the employee name is labeled as "name" rather than "employee". In this case, we can use the left_on and right_on keywords to specify the two column names:

In [107]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                   'salary': [7000, 8000, 12000, 900000]})

pd.merge(df1, df3, left_on="employee", right_on="name")

,employee,group,name,salary
0,Bob,Accounting,Bob,7000
1,Jake,Eng,Jake,8000
2,Lisa,Eng,Lisa,12000
3,Sue,HR,Sue,900000


The result has a redundant column that we can drop if desired–for example, by using the drop() method of DataFrames:

In [108]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,7000
1,Jake,Eng,8000
2,Lisa,Eng,12000
3,Sue,HR,900000


## Specifying Set Arithmetic for Joins

In all the preceding examples we have glossed over one important consideration in performing a join: the type of set arithmetic used in the join. 

This comes up when a value appears in one key column but not the other. Consider this example:

In [109]:
df4 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                   'food': ['fish', 'beans', 'bread']},
                  columns=['name', 'food'])
df4

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


In [110]:
df5 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                   'drink': ['wine', 'beer']},
                  columns=['name', 'drink'])
df5

,name,drink
0,Mary,wine
1,Joseph,beer


Here we have merged two datasets that have only a single "name" entry in common: Mary. By default, the result contains the intersection of the two sets of inputs; this is what is known as an inner join.

In [111]:
pd.merge(df4, df5)

,name,food,drink
0,Mary,bread,wine


An outer join returns a join over the union of the input columns, and fills in all missing values with NAs:

In [112]:
pd.merge(df4, df5, how='outer')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine
3,Joseph,NaN,beer


The left join and right join return joins over the left entries and right entries, respectively. For example:

In [113]:
pd.merge(df4, df5, how='left')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


The output rows now correspond to the entries in the left input. Using how='right' works in a similar manner.


In [114]:
pd.merge(df4, df5, how='right')

,name,food,drink
0,Mary,bread,wine
1,Joseph,NaN,beer


# 8. Aggregation and Grouping

An essential piece of analysis of large data is efficient summarization: computing aggregations like sum(), mean(), median(), min(), and max(), in which a single number gives insight into the nature of a potentially large dataset.

## Titanic Data

Here we will use the Titanic dataset, available via the Seaborn package.

This contains a wealth of information on each passenger:
* survival - Survival (0 = No; 1 = Yes)
* class - Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
* name - Name
* sex - Sex
* age - Age
* sibsp - Number of Siblings/Spouses Aboard
* parch - Number of Parents/Children Aboard
* ticket - Ticket Number
* fare - Passenger Fare
* cabin - Cabin
* embarked - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
* boat - Lifeboat (if survived)
* body - Body number (if did not survive and body was recovered)

In [115]:
import seaborn as sns
titanic = sns.load_dataset('titanic')
titanic.shape

(891, 15)

In [116]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [117]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            714 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


## Simple Aggregation in Pandas

We already explored some of the data aggregations available for NumPy array. We can do the same for pandas Series and DataFrames.

In addition, there is a convenience method describe() that computes several common aggregates for each column and returns the result. Let's use this on the Planets data, for now dropping rows with missing values:

In [118]:
titanic.dropna().describe()

,survived,pclass,age,sibsp,parch,fare
count,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000
mean,0.675824,1.192308,35.623187,0.467033,0.478022,78.919735
std,0.469357,0.516411,15.671615,0.645007,0.755869,76.490774
min,0.000000,1.000000,0.920000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,24.000000,0.000000,0.000000,29.700000
50%,1.000000,1.000000,36.000000,0.000000,0.000000,57.000000
75%,1.000000,1.000000,47.750000,1.000000,1.000000,90.000000
max,1.000000,3.000000,80.000000,3.000000,4.000000,512.329200


In [119]:
titanic.max()

survived            1
pclass              3
sex              male
age                80
sibsp               8
parch               6
fare          512.329
class           Third
who             woman
adult_male       True
alive             yes
alone            True
dtype: object

In [120]:
titanic['survived'].sum()

342

In [121]:
titanic['age'].mean()

29.69911764705882

In [122]:
# count of passangers in third class
titanic[titanic['class'] == 'Third']['class'].count()

491

## GroupBy

Simple aggregations can give you a flavor of your dataset, but often we would prefer to aggregate conditionally on some label or index: this is implemented in the so-called groupby operation. The name "group by" comes from a command in the SQL database language.


A groupby operation involves some combination of splitting the object, applying a function, and combining the results. This can be used to group large amounts of data and compute operations on these groups.

In [123]:
titanic['embark_town'].unique()

array(['Southampton', 'Cherbourg', 'Queenstown', nan], dtype=object)

The GroupBy object supports column indexing in the same way as the DataFrame, and returns a modified GroupBy object. For example:

In [124]:
titanic.groupby('embark_town')

Here we've selected a particular Series group from the original DataFrame group by reference to its column name. As with the GroupBy object, no computation is done until we call some aggregate on the object:

In [125]:
# survided passangers by class
titanic.groupby('embark_town')['survived'].sum()

embark_town
Cherbourg       93
Queenstown      30
Southampton    217
Name: survived, dtype: int64

In [126]:
# survided passangers by class
titanic.groupby('class')['survived'].sum()

class
First     136
Second     87
Third     119
Name: survived, dtype: int64

# Pivot Tables

The pivot table takes simple column-wise data as input, and groups the entries into a two-dimensional table that provides a multidimensional summarization of the data.

The difference between pivot tables and GroupBy can sometimes cause confusion; it helps me to think of pivot tables as essentially a multidimensional version of GroupBy aggregation. That is, you split-apply-combine, but both the split and the combine happen across not a one-dimensional index, but across a two-dimensional grid.

## Pivot Tables by Hand

To start learning more about this data, we might begin by grouping according to gender, survival status, or some combination thereof.

If you have read the previous section, you might be tempted to apply a GroupBy operation–for example, let's look at survival rate by gender:

In [127]:
# mean of survived passangers by sex
titanic.groupby('sex')[['survived']].mean()

,survived
sex,
female,0.742038
male,0.188908


This immediately gives us some insight: overall, three of every four females on board survived, while only one in five males survived!


This is useful, but we might like to go one step deeper and look at survival by both sex and, say, class. 

We group by class and gender, select survival, apply a mean aggregate, combine the resulting groups, and then unstack the hierarchical index to reveal the hidden multidimensionality. In code:

In [128]:
# go one step deeper and look at survival by both sex and, say, class.
titanic.groupby(['sex', 'class'])['survived'].aggregate('mean').unstack()

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


This gives us a better idea of how both gender and class affected survival, but the code is starting to look a bit garbled. 

Here is the equivalent to the preceding operation using the pivot_table method of DataFrames:

In [129]:
titanic.pivot_table('survived', index='sex', columns='class')

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


This is eminently more readable than the groupby approach, and produces the same result. 

## Multi-level pivot tables

The grouping in pivot tables can be specified with multiple levels, and via a number of options. For example, we might be interested in looking at age as a third dimension.

In [130]:
# pd.cut to split the ages into two groups
age = pd.cut(titanic['age'], [0, 18, 80])
titanic.pivot_table('survived', ['sex', age], 'class')

class               First    Second     Third
sex    age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 80]  0.972973  0.900000  0.423729
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 80]  0.375000  0.071429  0.133663

# 9. Vectorized String Operations

One strength of Python is its relative ease in handling and manipulating string data. Pandas builds on this and provides a comprehensive set of vectorized string operations that become an essential piece of the type of munging required when working with (read: cleaning up) real-world data.

## Introducing Pandas String Operations

We saw in previous sections how tools like NumPy and Pandas generalize arithmetic operations so that we can easily and quickly perform the same operation on many array elements. For example:

For arrays of strings, NumPy does not provide such simple access, and thus you're stuck using a more verbose loop syntax:

In [131]:
data = ['peter', 'Paul', 'MARY', 'guIDO']
[s.capitalize() for s in data]

['Peter', 'Paul', 'Mary', 'Guido']

This is perhaps sufficient to work with some data, but it will break if there are any missing values. For example:

In [132]:
data = ['peter', 'Paul', None, 'MARY', 'gUIDO']
[s.capitalize() for s in data]

AttributeError: 'NoneType' object has no attribute 'capitalize'

Pandas includes features to address both this need for vectorized string operations and for correctly handling missing data via the str attribute of Pandas Series and Index objects containing strings. So, for example, suppose we create a Pandas Series with this data:

In [133]:
names = pd.Series(data)
names

0    peter
1     Paul
2     None
3     MARY
4    gUIDO
dtype: object

In [134]:
names.str.capitalize()

0    Peter
1     Paul
2     None
3     Mary
4    Guido
dtype: object

### Methods similar to Python string methods

In [135]:
names.str.lower()

0    peter
1     paul
2     None
3     mary
4    guido
dtype: object

In [136]:
names.str.len()

0    5.0
1    4.0
2    NaN
3    4.0
4    5.0
dtype: float64

In [137]:
names.str.startswith('M')

0    False
1    False
2     None
3     True
4    False
dtype: object

In [138]:
names.str[0:3]

0     pet
1     Pau
2    None
3     MAR
4     gUI
dtype: object

## Example: Recipe Database

These vectorized string operations become most useful in the process of cleaning up messy, real-world data. Here I'll walk through an example of that, using an open recipe database compiled from various sources on the Web.

In [139]:
#!curl -O https://s3.amazonaws.com/openrecipes/20170107-061401-recipeitems.json.gz
#!gunzip 20170107-061401-recipeitems.json

In [140]:
recipes = pd.read_json('20170107-061401-recipeitems.json',  lines=True)


In [141]:
recipes.shape

(173278, 17)

We see there are nearly 200,000 recipes, and 17 columns. Let's take a look at one row to see what we have:

In [142]:
recipes.head()

,_id,cookTime,creator,dateModified,datePublished,description,image,ingredients,name,prepTime,recipeCategory,recipeInstructions,recipeYield,source,totalTime,ts,url
0,{'$oid': '5160756b96cc62079cc2db15'},PT30M,NaN,NaN,2013-03-11,"Late Saturday afternoon, after Marlboro Man ha...",http://static.thepioneerwoman.com/cooking/file...,Biscuits\n3 cups All-purpose Flour\n2 Tablespo...,Drop Biscuits and Sausage Gravy,PT10M,NaN,NaN,12,thepioneerwoman,NaN,{'$date': 1365276011104},http://thepioneerwoman.com/cooking/2013/03/dro...
1,{'$oid': '5160756d96cc62079cc2db16'},PT20M,NaN,NaN,2013-03-13,"When I was growing up, I participated in my Ep...",http://static.thepioneerwoman.com/cooking/file...,12 whole Dinner Rolls Or Small Sandwich Buns (...,Hot Roast Beef Sandwiches,PT20M,NaN,NaN,12,thepioneerwoman,NaN,{'$date': 1365276013902},http://thepioneerwoman.com/cooking/2013/03/hot...
2,{'$oid': '5160756f96cc6207a37ff777'},NaN,NaN,NaN,2013-01-07,A beauty of a carrot salad - tricked out with ...,http://www.101cookbooks.com/mt-static/images/f...,Dressing:\n1 tablespoon cumin seeds\n1/3 cup /...,Morrocan Carrot and Chickpea Salad,PT15M,NaN,NaN,NaN,101cookbooks,NaN,{'$date': 1365276015332},http://www.101cookbooks.com/archives/moroccan-...
3,{'$oid': '5160757096cc62079cc2db17'},PT15M,NaN,NaN,2013-03-18,It's Monday! It's a brand new week! The birds ...,http://static.thepioneerwoman.com/cooking/file...,Biscuits\n3 cups All-purpose Flour\n2 Tablespo...,Mixed Berry Shortcake,PT15M,NaN,NaN,8,thepioneerwoman,NaN,{'$date': 1365276016700},http://thepioneerwoman.com/cooking/2013/03/mix...
4,{'$oid': '5160757496cc6207a37ff778'},NaN,NaN,NaN,2013-01-20,A simple breakfast bowl made with Greek yogurt...,http://www.101cookbooks.com/mt-static/images/f...,For each bowl: \na big dollop of Greek yogurt\...,Pomegranate Yogurt Bowl,PT5M,NaN,NaN,Serves 1.,101cookbooks,NaN,{'$date': 1365276020318},http://www.101cookbooks.com/archives/pomegrana...


In [143]:
recipes.iloc[0]

_id                                {'$oid': '5160756b96cc62079cc2db15'}
cookTime                                                          PT30M
creator                                                             NaN
dateModified                                                        NaN
datePublished                                                2013-03-11
description           Late Saturday afternoon, after Marlboro Man ha...
image                 http://static.thepioneerwoman.com/cooking/file...
ingredients           Biscuits\n3 cups All-purpose Flour\n2 Tablespo...
name                                    Drop Biscuits and Sausage Gravy
prepTime                                                          PT10M
recipeCategory                                                      NaN
recipeInstructions                                                  NaN
recipeYield                                                          12
source                                                  thepione

There is a lot of information there, but much of it is in a very messy form, as is typical of data scraped from the Web. In particular, the ingredient list is in string format; we're going to have to carefully extract the information we're interested in. Let's start by taking a closer look at the ingredients:

In [144]:
recipes.ingredients.str.len().describe()

count    173278.000000
mean        247.425992
std         147.080372
min           0.000000
25%         151.000000
50%         224.000000
75%         317.000000
max        9083.000000
Name: ingredients, dtype: float64

The ingredient lists average 250 characters long, with a minimum of 0 and a maximum of nearly 10,000 characters!

Just out of curiousity, let's see which recipe has the longest ingredient list:

In [145]:
recipes.name[np.argmax(recipes.ingredients.str.len())]

'Carrot Pineapple Spice &amp; Brownie Layer Cake with Whipped Cream &amp; Cream Cheese Frosting and Marzipan Carrots'

We can do other aggregate explorations; for example, let's see how many of the recipes are for breakfast food:

In [146]:
recipes.description.str.contains('[Bb]reakfast').sum()

3524

Or how many of the recipes list cinnamon as an ingredient:



In [147]:
recipes.ingredients.str.contains('[Cc]innamon').sum()

10526

## A simple recipe recommender

Let's go a bit further, and start working on a simple recipe recommendation system: given a list of ingredients, find a recipe that uses all those ingredients. While conceptually straightforward, the task is complicated by the heterogeneity of the data: there is no easy operation, for example, to extract a clean list of ingredients from each row. So we will cheat a bit: we'll start with a list of common ingredients, and simply search to see whether they are in each recipe's ingredient list. For simplicity, let's just stick with herbs and spices for the time being:

In [148]:
ingredient_list = ['chicken', 'potatoes', 'parsley', 'beer']

In [149]:
import re
ingredient_df = pd.DataFrame(dict((ingredient, recipes.ingredients.str.contains(ingredient, re.IGNORECASE))
                            for ingredient in ingredient_list))

ingredient_df

,beer,chicken,parsley,potatoes
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
5,False,False,False,False
6,False,False,False,False
7,False,False,False,False
8,False,False,False,False
9,False,False,False,False


Now, as an example, let's say we'd like to find a recipe that uses parsley, paprika, and sage. We can compute this very quickly using the query() method of DataFrames:

In [150]:
selection = ingredient_df.query('chicken & potatoes & beer')
len(selection)

16

In [151]:
recipes.name[selection.index]

1364                                       Bangers and Mash
91575                            Beer Butt Rosemary Chicken
91601                                  Beer Cheese Soup III
93650                               Bubbly Beer Cheese Soup
96831                                    Chicken With Stout
101334                                Dark Beer Cheese Soup
109214                                      Hot Sausage Pot
118284                                   Peruvian Lamb Soup
124534                           Slow Drunk Roasted Chicken
128676                                 Terry's Beer Chicken
137872                Roast Chicken with Smoky Potato Salad
140729    Grill-Roasted Chicken with Potatoes &amp; Gree...
142123    Grill-Roasted Chicken with Potatoes and Bell P...
150319     Fish and chips with tartare sauce and pea purÃ©e
156747                          Fish, chips and curry sauce
159232    Braised pork cheeks with ginger carrots and Je...
Name: name, dtype: object